In [6]:
import torch
import numpy as np
from torch import nn,optim
from torch.utils.data import Dataset, DataLoader
import pyreadr
import torch.nn.functional as F
import matplotlib.pyplot as plt 
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas as pd

In [7]:
# check pytorch version
print(torch.__version__)

1.4.0


In [22]:
# Define the class to get our dataset.
class Data(Dataset):
    def __init__(self, Dataset, x, y):
        self.x=torch.FloatTensor(Dataset[pred_vars].values)
        self.y=torch.FloatTensor(Dataset[y].values)
        self.y=self.y.view(-1,1)
        self.len=self.x.shape[0]
    def __getitem__(self,index):         
        return self.x[index],self.y[index]
    def __len__(self):
        return self.len

In [9]:
def normalize(df, parameters = None):
    result = df.copy()
    if parameters is None:
        frame = { 'mean': df.mean(), 'std': df.std()} 
        parameters = pd.DataFrame(frame) 
        parameters.reset_index(inplace=True) # Resets the index, makes factor a column   
        
    for feature_name in df.columns:
        mean_value = parameters.loc[(parameters['index'] == feature_name), 'mean']
        std_value = parameters.loc[(parameters['index'] == feature_name), 'std']
        result[feature_name] = (df[feature_name] - mean_value.values)/std_value.values
    return [result, parameters]

In [30]:
# Get our training dataset.
pred_vars = pyreadr.read_r("/home/germanm2/n_policy_box/Data/files_rds/pred_vars.rds")[None] # also works for RData
pred_vars = [item for sublist in pred_vars.values.tolist()  for item in sublist]

TrainSet_eonr2_df = pyreadr.read_r("/home/germanm2/n_policy_box/Data/files_rds/TrainSet_eonr2.rds")[None] # also works for RData
data_and_parameters = normalize(TrainSet_eonr2_df)
data_set = data_and_parameters[0]
parameters_training = data_and_parameters[1]
data_set2=Data(data_set, x = pred_vars, y = 'eonr')

In [34]:
# Get our validation dataset.
prediction_set_aggregated_df = pyreadr.read_r("/home/germanm2/n_policy_box/Data/files_rds/prediction_set_aggregated_dt.rds")[None] # also works for RData
prediction_set_aggregated_df = prediction_set_aggregated_df.rename(columns={"eonr_12": "eonr"}) #needed for tha Date class
# validation_set = scaler.transform(prediction_set_aggregated_df[pred_vars+['eonr']])
# validation_set2 = Data(validation_set, x=pred_vars, y='eonr')
validation_set = normalize(TrainSet_eonr2_df, parameters = parameters_training)[0]
validation_set
validation_set2 = Data(validation_set, x=pred_vars, y='eonr')

In [42]:
def normalize_back(df, parameters = None):
    df = validation_set
    result = df.copy()
    for feature_name in df.columns:
        mean_value = parameters.loc[(parameters['index'] == feature_name), 'mean']
        std_value = parameters.loc[(parameters['index'] == feature_name), 'std']
        result[feature_name] = (df[feature_name]  * std_value.values) + mean_value.values
    return result

In [43]:
normalize_back(validation_set, parameters = parameters_training)

,eonr,rain_30,rain_60,rain_90,t_max_30,t_max_60,t_max_90,t_min_30,t_min_60,t_min_90,...,day_sow,day_v5,lai_v5,whc,oc_20cm_v5,sw_dep_v5,n_0_60cm_v5,surfaceom_wt_v5,sand_40cm,clay_40cm
0,210.0,94.0,191.0,90.0,22.766667,15.550000,5.433333,10.383333,4.183333,-2.783333,...,111.0,147.0,0.528,288.0,2.03525,637.087,32.7,310.8,4.000000,24.262000
1,230.0,124.0,123.0,17.0,22.033333,15.050000,12.100000,9.783333,1.783333,-0.766667,...,111.0,150.0,0.570,288.0,2.03500,659.419,38.3,370.4,4.000000,24.262000
2,240.0,271.0,98.0,46.0,19.866667,18.083333,7.466667,8.033333,5.816667,-3.550000,...,111.0,157.0,0.472,288.0,2.03525,728.974,40.3,318.9,4.000000,24.262000
3,200.0,151.0,69.0,113.0,18.850000,15.216667,9.250000,7.816667,2.733333,-0.466667,...,111.0,152.0,0.532,288.0,2.03575,647.879,30.5,330.7,4.000000,24.262000
4,140.0,18.0,112.0,32.0,22.183333,14.066667,7.183333,6.850000,3.250000,-3.183333,...,111.0,154.0,0.186,288.0,2.03350,624.983,57.0,610.6,4.000000,24.262000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4258,120.0,259.0,143.0,62.0,23.866667,13.933333,11.500000,11.216667,1.916667,-1.616667,...,95.0,128.0,0.537,294.5,1.16825,659.938,9.8,258.2,7.655814,18.032558
4259,300.0,201.0,124.0,66.0,22.266667,13.500000,13.900000,10.033333,1.283333,2.950000,...,95.0,136.0,0.526,294.5,1.16825,650.698,10.5,263.5,7.655814,18.032558
4260,210.0,65.0,115.0,61.0,21.966667,13.066667,15.350000,9.100000,0.916667,3.500000,...,95.0,131.0,0.330,294.5,1.16700,589.504,32.2,448.5,7.655814,18.032558
4261,180.0,92.0,108.0,73.0,22.733333,16.000000,11.250000,9.100000,2.750000,-1.350000,...,95.0,133.0,0.543,294.5,1.16775,599.784,16.8,342.0,7.655814,18.032558


In [ ]:
# # Create the class for model 
# class Net(torch.nn.Module):
#     def __init__(self, cols):
#         super(Net, self).__init__()
#         self.hidden1 = torch.nn.Linear(cols, 21)   # hidden layer
#         self.hidden2 = torch.nn.Linear(21, 10)   # hidden layer
#         self.predict = torch.nn.Linear(10, 1)   # output layer

#     def forward(self, x):
#         x = F.relu(self.hidden1(x))      # activation function for hidden layer
#         x = F.relu(self.hidden2(x))      # activation function for hidden layer
#         x = self.predict(x)             # linear output
#         return x
    
# cols = len(pred_vars)
# model = Net(cols)

In [ ]:
# Create the class for model (with Dropout)
class Net(nn.Module):
    # Constructor
    def __init__(self, cols, p=0.1):
        super(Net, self).__init__()
        self.drop = nn.Dropout(p=p)
        self.hidden1 = torch.nn.Linear(cols, 30)   # hidden layer
        self.hidden2 = torch.nn.Linear(30, 5)   # hidden layer
        self.hidden3 = torch.nn.Linear(5, 5)   # hidden layer
        self.hidden4 = torch.nn.Linear(5, 5)   # hidden layer
        self.predict = torch.nn.Linear(5, 1)   # output layer

    def forward(self, x):
        x = F.relu(self.drop(self.hidden1(x)))      # activation function for hidden layer
        x = F.relu(self.drop(self.hidden2(x)))      # activation function for hidden layer
        x = F.relu(self.drop(self.hidden3(x)))      # activation function for hidden layer
        x = F.relu(self.drop(self.hidden4(x)))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x

In [ ]:
# # Create the class for model (simple)
# class Net(torch.nn.Module):
#     def __init__(self, cols):
#         super(Net, self).__init__()
#         self.hidden1 = torch.nn.Linear(cols, 1)   # hidden layer
#         self.predict = torch.nn.Linear(1, 1)   # output layer

#     def forward(self, x):
#         x = F.relu(self.hidden1(x))      # activation function for hidden layer
#         x = self.predict(x)             # linear output
#         return x
    
# cols = len(pred_vars)
# model = Net(cols)

In [ ]:
# # Make one prediction
# x = training_set[0][0]
# y = training_set[0][0]
# print(model.state_dict())
# print(model(x))


In [ ]:
#  Create the function to train our model, which accumulate lost for each iteration to obtain the cost.
def train_mini_batches(training_set, train_loader, validation_set, model,criterion, optimizer, epochs=5, plot_epoch = False):
    cost_training=[] 
    cost_validation=[]
    for epoch in range(epochs):
        total = 0

        for x,y in train_loader:
            optimizer.zero_grad()
            
            yhat=model(x)
            loss=criterion(yhat,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total+=loss.item()
        cost_training.append(total)
        cost_validation.append(criterion(model(validation_set.x),validation_set.y))

    if plot_epoch:
        fig, ax1 = plt.subplots()
        color = 'tab:red'
        ax1.set_xlabel('epoch)')
        ax1.set_ylabel('cost', color=color)
        ax1.plot(cost_training, color=color, label='training') 
        ax1.tick_params(axis='y', labelcolor=color)
        ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
        color = 'tab:blue'
        ax2.set_ylabel('cost', color=color)  # we already handled the x-label with ax1
        ax2.plot(cost_validation, color=color, label='validation') 
        ax2.tick_params(axis='y', labelcolor=color)
        fig.legend()
        fig.tight_layout()  # otherwise the right y-label is slightly clipped
        plt.show()

In [ ]:
def train(training_set2, validation_set2, model,criterion, optimizer, learning_rate, epochs=5, plot_epoch = False):
    cost_training=[] 
    cost_validation=[]  
    for epoch in range(epochs):
         #all the samples are used for training because they fit in memory
        optimizer.zero_grad()
        yhat = model(training_set2.x)
        loss=criterion(yhat,training_set2.y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        cost_training.append(loss)
        cost_validation.append(criterion(model(validation_set2.x),validation_set2.y))
#         if epoch > 500: #this will lower the learning rate at the end of the training
#             optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate/10)
    if plot_epoch:
        fig, ax1 = plt.subplots()
        color = 'tab:red'
        ax1.set_xlabel('epoch)')
        ax1.set_ylabel('cost', color=color)
        ax1.plot(cost_training, color=color, label='training') 
        ax1.tick_params(axis='y', labelcolor=color)
        ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
        color = 'tab:blue'
        ax2.set_ylabel('cost', color=color)  # we already handled the x-label with ax1
        ax2.plot(cost_validation, color=color, label='validation') 
        ax2.tick_params(axis='y', labelcolor=color)
        fig.legend()
        fig.tight_layout()  # otherwise the right y-label is slightly clipped
        plt.show()
#     plt.figure()
#     plt.plot(cost_training, label='training') 
#     plt.plot(cost_validation, label='validation') 
#     plt.xlabel('epoch')
#     plt.ylabel('cost')
#     plt.legend()
#     plt.show()

In [ ]:
# # Get a single item from enumerate

# singleitem = next(iter(train_loader))
# x = singleitem[0]
# y = singleitem[1]
# yhat = model(x)

In [ ]:
# Create our model 
cols = len(pred_vars)
model = Net(cols, p=0.1)
model.train()

torch.manual_seed(0)
learning_rate = 0.01
criterion = torch.nn.MSELoss(reduction='mean')  # this is for regression mean squared loss 
# criterion = torch.nn.L1Loss() # this is for regression mean absolute loss 

optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

# train_loader=DataLoader(dataset=data_set,batch_size=32, shuffle=True)
# train_mini_batches(data_set, train_loader, validation_set, model,criterion, optimizer, epochs=100, plot_epoch = True)
train(training_set2, validation_set2, model,criterion, optimizer, learning_rate, epochs=1000, plot_epoch = True)

In [ ]:
prediction_set_aggregated_df = pyreadr.read_r("/home/germanm2/n_policy_box/Data/files_rds/prediction_set_aggregated_dt.rds")[None] # also works for RData
prediction_set_aggregated_df = prediction_set_aggregated_df.rename(columns={"eonr_12": "eonr"}) #needed for tha Date class
prediction_set = scaler.transform(prediction_set_aggregated_df[pred_vars+['eonr']])
prediction_set = np.delete(prediction_set, 20, 1)
prediction_set = torch.FloatTensor(prediction_set)

In [ ]:
y_pred = model(prediction_set)
y_pred

In [ ]:
#Make predictions with returned model
prediction_set_aggregated_df = pyreadr.read_r("/home/germanm2/n_policy_box/Data/files_rds/prediction_set_aggregated_dt.rds")[None] # also works for RData

X = Variable(torch.FloatTensor(prediction_set_aggregated_df[pred_vars].values)) 
model.eval()
y_pred = model(X) #This outputs the value for regression
y_pred=y_pred.data[:,0].numpy()

prediction_set_aggregated_df['eonr_pred'] = y_pred


plt.figure()
plt.plot(prediction_set_aggregated_df['eonr_12'], prediction_set_aggregated_df['eonr_pred'], 'o', color='black')
plt.xlabel('eonr_12')
plt.ylabel('eonr_pred_cnn')
plt.show()

plt.figure()
plt.plot(prediction_set_aggregated_df['eonr_12'], prediction_set_aggregated_df['eonr_pred_rf'], 'o', color='black')
plt.xlabel('eonr_12')
plt.ylabel('eonr_pred_rf')
plt.show()

plot_df = prediction_set_aggregated_df.groupby('eonr_12', as_index=False)['eonr_pred','eonr_pred_rf'].mean()
plt.figure()
plt.plot(plot_df['eonr_12'], plot_df['eonr_pred'], label='cnn') 
plt.plot(plot_df['eonr_12'], plot_df['eonr_pred_rf'], label='rf') 
plt.xlabel('eonr_12')
plt.ylabel('eonr_pred')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

print(prediction_set_aggregated_df['eonr_pred'].min(),
prediction_set_aggregated_df['eonr_pred'].max(),
prediction_set_aggregated_df['eonr_pred'].mean(),
mean_squared_error(prediction_set_aggregated_df['eonr_12'], prediction_set_aggregated_df['eonr_pred']),
mean_squared_error(prediction_set_aggregated_df['eonr_12'], prediction_set_aggregated_df['eonr_pred_rf']))

In [ ]:
4050 3472 (21, 21, 10, 1)  3322 (21,10,5,1) 2983 (10, 5, 5, 5, 1) 2167 2164 3004

In [ ]:
y = torch.tensor([100,160],dtype=torch.float32)
yhat= torch.tensor([110,170],dtype=torch.float32)
criterion1 = torch.nn.L1Loss()
criterion2 = torch.nn.MSELoss(reduction='mean')  # this is for regression mean squared loss 
print(criterion1(y, yhat), criterion2(y, yhat))
criterion_up(y, yhat)

In [ ]:
def criterion_up(y, yhat):
    errors = yhat - y
    errors_subpred = errors[errors < 0]
    errors_overpred = errors[errors > 0]
    if errors_subpred.data.nelement()>0: #correction for zero length
        criterion_subpred = sum((errors_subpred)**2) / errors_subpred.data.nelement()
    else:
        criterion_subpred = torch.tensor([0],dtype=torch.float32)
    if errors_overpred.data.nelement()>0: #correction for zero length
        criterion_overpred = sum((errors_overpred)) / errors_overpred.data.nelement()
    else:
        criterion_overpred = torch.tensor([0],dtype=torch.float32)
    criterion = criterion_subpred.add(criterion_overpred) 
    criterion
    return(criterion)

In [ ]:
# yhat = model(data_set.x)
y = torch.tensor([100,160,170,130],dtype=torch.float32)
yhat= torch.tensor([90,170,150,150],dtype=torch.float32)

# loss=criterion_up(yhat,data_set.y)
errors = yhat - y
errors
errors_subpred = errors[errors < 0]
errors_overpred = errors[errors > 0]

if errors_subpred.data.nelement()>0: #correction for zero length
    criterion_subpred = sum((errors_subpred)**2) / errors_subpred.data.nelement()
else:
    criterion_subpred = torch.tensor([0],dtype=torch.float32)
if errors_overpred.data.nelement()>0: #correction for zero length
    criterion_overpred = sum((errors_overpred)) / errors_overpred.data.nelement()
else:
    criterion_overpred = torch.tensor([0],dtype=torch.float32)
criterion = criterion_subpred.add(criterion_overpred) 
criterion
yhat = model(data_set.x)
criterion = criterion_up
criterion(data_set.y,yhat)

In [ ]:
def build_cnn(TrainSet_eonr2_df, policy, pred_vars):
    #Define training hyperprameters.
    data_set=Data(TrainSet_eonr2_df, pred_vars)
    
    cols = len(pred_vars)

    # Create our model 
    model = Net(cols, p=0.1)
    model.train()

    torch.manual_seed(0)
    learning_rate = 0.01
    criterion = torch.nn.MSELoss(reduction='mean')  # this is for regression mean squared loss 
    # criterion = torch.nn.L1Loss() # this is for regression mean absolute loss 

    optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

    train(data_set, validation_set, model,criterion, optimizer, learning_rate, epochs=1000, plot_epoch = False)

    path = '/home/germanm2/n_policy_box/Data/files_rds/cnn_models/'+ policy + '.pth'
    torch.save(model.state_dict(), path)
    return(model)  
        

In [ ]:
policy = 'ratio_5'
pred_vars = pyreadr.read_r("/home/germanm2/n_policy_box/Data/files_rds/pred_vars.rds")[None] # also works for RData
pred_vars = [item for sublist in pred_vars.values.tolist()  for item in sublist]
TrainSet_eonr2_df = pyreadr.read_r("/home/germanm2/n_policy_box/Data/files_rds/TrainSet_eonr2.rds")[None] # also works for RData
net_return = build_cnn(TrainSet_eonr2_df, policy, pred_vars)

In [ ]:
#Make predictions with returned model
prediction_set_aggregated_df = pyreadr.read_r("/home/germanm2/n_policy_box/Data/files_rds/prediction_set_aggregated_dt.rds")[None] # also works for RData

X = Variable(torch.FloatTensor(prediction_set_aggregated_df[pred_vars].values)) 

y_pred = model(X) #This outputs the value for regression
y_pred=y_pred.data[:,0].numpy()

prediction_set_aggregated_df['eonr_pred'] = y_pred

In [ ]:
#Make predictions with returned model
prediction_set_aggregated_df = pyreadr.read_r("/home/germanm2/n_policy_box/Data/files_rds/prediction_set_aggregated_dt.rds")[None] # also works for RData

X_pred = prediction_set_aggregated_df[pred_vars]

X_pred=X_pred.values

X = Variable(torch.FloatTensor(X_pred)) 
y_pred = net_return(X) #This outputs the value for regression
y_pred=y_pred.data[:,0].numpy()
y_pred
prediction_set_aggregated_df['eonr_pred'] = y_pred

# now let's write a Rds
pyreadr.write_rds("/home/germanm2/n_policy_box/Data/files_rds/prediction_set_aggregated_cnn_dt.rds", prediction_set_aggregated_df)
prediction_set_aggregated_df.head()

In [ ]:
#Load the saved model
policy = 'ratio_5'
path = '/home/germanm2/n_policy_box/Data/files_rds/cnn_models/'+ policy + '.pth'
net_load = Net(21, 100, 1)
net_load.load_state_dict(torch.load(path))
net_load.eval()
net_load.state_dict()


In [ ]:
#Make predictions with the saved model
prediction_set_aggregated_df = pyreadr.read_r("/home/germanm2/n_policy_box/Data/files_rds/prediction_set_aggregated_dt.rds")[None] # also works for RData

X_pred = prediction_set_aggregated_df[pred_vars]
X_pred=X_pred.values
X_pred
X = Variable(torch.FloatTensor(X_pred)) 
y_pred = net_load(X) #This outputs the value for regression
y_pred=y_pred.data[:,0].numpy()
y_pred
prediction_set_aggregated_df['eonr_pred'] = y_pred

# now let's write a Rds
pyreadr.write_rds("/home/germanm2/n_policy_box/Data/files_rds/prediction_set_aggregated_cnn_dt.rds", prediction_set_aggregated_df)
prediction_set_aggregated_df.head()

In [ ]:
# Make a function that loads the saved model and does predictions

prediction_set_aggregated_df = pyreadr.read_r("/home/germanm2/n_policy_box/Data/files_rds/prediction_set_aggregated_dt.rds")[None]
prediction_set_aggregated_df.head()

In [ ]:
def predict_cnn(prediction_set_aggregated_df, policy, pred_vars):
    #Load the saved model
    #policy = 'ratio_5'
    cols = len(pred_vars)
    model_load = Net(cols)
    path = '/home/germanm2/n_policy_box/Data/files_rds/cnn_models/'+ policy + '.pth'
    model_load.load_state_dict(torch.load(path))
    model_load.eval()
    model_load.state_dict()
    X_pred = prediction_set_aggregated_df[pred_vars]
    X_pred=X_pred.values
    X_pred
    X = Variable(torch.FloatTensor(X_pred)) 
    y_pred = model_load(X) #This outputs the value for regression
    y_pred=y_pred.data[:,0].numpy()
    y_pred
    prediction_set_aggregated_df['eonr_pred'] = y_pred
    return(prediction_set_aggregated_df)

In [ ]:
# Use the function
prediction_set_aggregated_df2 = predict_cnn(prediction_set_aggregated_df, 'ratio_5', pred_vars)
prediction_set_aggregated_df2

In [ ]:
from sklearn.metrics import mean_squared_error

print(prediction_set_aggregated_df['eonr_pred'].min(),
prediction_set_aggregated_df['eonr_pred'].max(),
prediction_set_aggregated_df['eonr_pred'].mean(),
mean_squared_error(prediction_set_aggregated_df['eonr_12'], prediction_set_aggregated_df['eonr_pred']))

In [ ]:
# 3088
import matplotlib.pyplot as plt
import numpy as np

# Fixing random state for reproducibility
np.random.seed(19680801)


x = prediction_set_aggregated_df['eonr_12']
y = prediction_set_aggregated_df['eonr_pred']

plt.scatter(x, y, c="g", alpha=0.5, marker=r'$\clubsuit$',
            label="Luck")
plt.legend(loc='upper left')
plt.show()